In [4]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
import json
from sklearn.metrics.pairwise import  cosine_similarity
from scipy import sparse
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
from random import seed

In [40]:
# Create an engine instance
with open("../dags/ops/config/config.json", "r") as config:
    print('Reading config file')
    configuration = json.loads(config.read())
alchemyEngine = create_engine(f'postgresql+psycopg2://{configuration["db_connection"]["user"]}:{configuration["db_connection"]["password"]}@{configuration["db_connection"]["host"]}:{configuration["db_connection"]["port"]}/{configuration["db_connection"]["database"]}',
                                 pool_recycle=3600)

dbConnection = alchemyEngine.connect()
# Select all the products that have not been rated by the user. Example for customer_id = 1684
df = pd.read_sql('select * from ratings', dbConnection)
products = pd.read_sql(f'''select p.product_id, p.product_name, pc.product_category_name 
from (
	select r.product_id, r.rating 
	from ratings r
	where customer_id = {1684}
	) A
	right outer join product p 
		on A.product_id = p.product_id
	left join product_category pc 
		on pc.product_category_id = p.product_category_id 
where A.product_id is null''', dbConnection)
print(df.head())
print(products.head())
dbConnection.close()

Reading config file
   rating_id  customer_id  product_id  rating  rating_dt
0       9972         1684          40       5 2020-02-18
1       9973         1684          45       5 2020-02-12
2       9974         1684          49       5 2020-01-22
3       9975         1684          59       5 2020-01-29
4       9976         1684          60       2 2020-02-02
   product_id     product_name product_category_name
0          63         Playfoam          Art & Crafts
1          62  Playdoh Toolkit          Art & Crafts
2          61  Playdoh Playset          Art & Crafts
3          54       Magic Sand          Art & Crafts
4          52  Kids Makeup Kit          Art & Crafts


In [8]:
# Get relevant fields
df = df.loc[:,["customer_id","product_id","rating"]]
print(df.head())

   customer_id  product_id  rating
0         1684          40       5
1         1684          45       5
2         1684          49       5
3         1684          59       5
4         1684          60       2


In [34]:
# Split the data into train and test
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size = 0.3)
print(train.head())
print(test.head())

      customer_id  product_id  rating
8440         3104          45       5
1928         2011          49       2
6568         2790          43       5
4842         2500          66       3
8425         3101          62       1
      customer_id  product_id  rating
6131         2716          56       4
9605         3302          40       3
2790         2154          49       2
6175         2725          49       1
7963         3023          56       5


In [35]:
# For Item based Collaborative Filter, 
# we can use Single Value Decomposition (SVD) 
# to predict the rating on an unseen item for a particular user.
# We could try similarity between userr or items. However, those
# models have disadvantages and are not scalable as SVD. 
# Therefore, we decide to use this one
reader = Reader()
data = Dataset.load_from_df(train, reader)
svd = SVD()
cross_validate(svd, data, measures=["RMSE","MAE"], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.4313  1.4426  1.4560  1.4385  1.4501  1.4437  0.0086  
MAE (testset)     1.2186  1.2353  1.2544  1.2320  1.2392  1.2359  0.0115  
Fit time          0.49    0.85    0.45    0.52    0.48    0.56    0.15    
Test time         0.01    0.01    0.02    0.01    0.01    0.01    0.00    


{'test_rmse': array([1.43130652, 1.44256317, 1.45599916, 1.43850651, 1.4500861 ]),
 'test_mae': array([1.21861631, 1.2352913 , 1.25435272, 1.23202897, 1.23916469]),
 'fit_time': (0.493009090423584,
  0.8485326766967773,
  0.44692516326904297,
  0.5203361511230469,
  0.47887206077575684),
 'test_time': (0.008405685424804688,
  0.013129472732543945,
  0.015207290649414062,
  0.012772560119628906,
  0.014789342880249023)}

In [36]:
# Fit the model for the trainset
trainset = data.build_full_trainset()
svd.fit(trainset)

In [37]:
import pickle

# Save the model
with open('svd','wb') as file:
    pickle.dump(svd,file)

In [42]:
# ================== Prediction =====================
# Load the model to make predictions
with open('svd','rb') as file:
    svd = pickle.load(file)
    print('model loaded')

# Step 1. Input data
# data = {'customer_id':  [1,1,1,1,1,1,1,1,1,1],
#         'product_id': [70,69,68,67,66,65,64,63,62,61]
#         }
# prediction = pd.DataFrame(data)

# Test with the products for user 1684
data = products.copy()
data["customer_id"] = 1684
data.head()

# Prediction on score
for i, j in prediction.iterrows():
    result = svd.predict(uid=j["customer_id"],iid=j["product_id"])
    #if result.est >=3.5:
    print(result)

    

model loaded
user: 1          item: 70         r_ui = None   est = 3.33   {'was_impossible': False}
user: 1          item: 69         r_ui = None   est = 2.95   {'was_impossible': False}
user: 1          item: 68         r_ui = None   est = 2.97   {'was_impossible': False}
user: 1          item: 67         r_ui = None   est = 3.02   {'was_impossible': False}
user: 1          item: 66         r_ui = None   est = 2.95   {'was_impossible': False}
user: 1          item: 65         r_ui = None   est = 3.08   {'was_impossible': False}
user: 1          item: 64         r_ui = None   est = 3.02   {'was_impossible': False}
user: 1          item: 63         r_ui = None   est = 3.02   {'was_impossible': False}
user: 1          item: 62         r_ui = None   est = 3.00   {'was_impossible': False}
user: 1          item: 61         r_ui = None   est = 3.02   {'was_impossible': False}
